In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [9]:

if __name__ == '__main__':

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"
    
    # BrightData proxy credentials
    PROXY_USERNAME = 'brd-customer-hl_4ae3c379-zone-web_unlocker1'
    PROXY_PASSWORD = 'nzvr18y7otsh'
    PROXY_HOST = 'brd.superproxy.io'  # e.g., 'zproxy.lum-superproxy.io'
    PROXY_PORT = '22225'  # e.g., '22225'

    # Informations du proxy
    proxy = f'http://{PROXY_USERNAME}:{PROXY_PASSWORD}@{PROXY_HOST}:{PROXY_PORT}'
    proxies = {
        'http': proxy,
        'https': proxy,
    }

    # Paramètres pour contourner les vérifications SSL
    session = requests.Session()
    session.proxies.update(proxies)
    session.verify = False  # Contourner la vérification SSL
    
    # Faire une requête
    webpage = session.get(URL)
    #response.raise_for_status()  # Vérifier si la requête a réussi

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = session.get("https://www.amazon.com" + link)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


C:\Users\lenovo\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\lenovo\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\lenovo\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brd.superproxy.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\lenovo\anaconda3\Lib\site-packages\urlli

In [13]:
amazon_df.head()

,title,price,rating,reviews,availability
0,PlayStation 4 500GB Console (Renewed),,4.5 out of 5 stars,"1,300 ratings",In Stock
1,Sony PlayStation 4 Slim Limited Edition 1TB Ga...,,4.5 out of 5 stars,"1,467 ratings",Only 7 left in stock - order soon.
2,PlayStation 4 Slim 1TB Console - Black (Renewed),,4.5 out of 5 stars,"1,701 ratings",Only 10 left in stock - order soon.
3,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,357 ratings,Only 7 left in stock - order soon.
4,PlayStation®4 Console – Call of Duty® Modern W...,,4.5 out of 5 stars,108 ratings,In Stock
